<center>Progetto realizzato da Elena Curti (matr. 185431)

# Recipes
</center>

## Requisiti
Questo progetto è stato realizzato con Python 3.11 e Neo4j. E' necessaria l'installazione dei seguenti pacchetti:

In [1]:
# %pip install py2neo neo4j-driver

## Operazioni iniziali
Per il corretto funzionamento del progetto proposto, è necessario creare il DBMS e importarci i dati, seguendo i seguenti passi. <br>
Da Neo4j creare un nuovo progetto. Scegliere una tra le seguenti alternative:
- Dal progetto appena creato, aggiungere al progetto il file [neo4j.dump](data/neo4j.dump). Creare poi un nuovo DBMS dal file dump (scegliendo come nome "neo4j", come password "pass" e come versione 5.2.0). Aggiungere il plugin APOC. Premere su Start.
- Dal progetto appena creato, aggiungere un DBMS locale (scegliendo come nome, password e versione i valori indicati al punto sopra). Aggiungere il plugin APOC. Premere su Start e poi su Open. Eseguire tutti i comandi riportati nel file [import_commands.txt](data/import_commands.txt).
<br>
Eseguire poi le celle di codice, nell'ordine proposto

## Connessione e autenticazione

In [3]:
from py2neo import Graph
graph = Graph("bolt://localhost:7687",  auth=("neo4j", "pass"))

print("Connessione al database riuscita! \nEcco 5 ricette d'esempio:")
cq = "MATCH (p:Recipe) RETURN p.name LIMIT 5"
for rec in graph.run(cq):
    print("\t",str(rec)[1:-1])

Connessione al database riuscita! 
Ecco 5 ricette d'esempio:
	 Tomato & mozzarella couscous salad
	 Baked nectarines & raspberries with almonds & honey
	 Chocolate & caramel ombre cake
	 Chicken, goat's cheese & cherry tomato bake
	 Chipotle chicken tacos with pineapple salsa


# Schema
Di seguito è riportata la visualizzazione sottoforma di grafo del comando:
```
    call db.schema.visualization()
```

![schema](img/db_schema2.png)

## Aggiunta degli indici
Per migliorare la performance delle cypher query sono stati aggiunti i seguenti indici: 
    <ul>
        <li><i>id</i> su Recipe</li>
        <li><i>name</i> su Ingredient</li>
        <li><i>name</i> su Keyword</li>
        <li><i>name</i> su DietType</li>
        <li><i>name</i> su Author</li>
        <li><i>name</i> su Collection</li>
    </ul>
Per creare gli indici è stato usato il seguente script. Il comando ```CREATE INDEX IF NOT EXISTS FOR (n:Recipe) ON (n.id)``` è equivalente a ```CREATE INDEX IF NOT EXISTS ON :Recipe(name)```. Il primo, però, è compatibile con la versione 5.2.0 di Neo4j.

In [4]:
def run_multiple_query(multiple_query):
    for query in multiple_query.splitlines():
        if query == "": continue
        graph.run(query)

cq_versione5= """
CREATE INDEX IF NOT EXISTS FOR (n:Recipe) ON (n.id);        // CREATE INDEX IF NOT EXISTS ON :Recipe(name) 
CREATE INDEX IF NOT EXISTS FOR (n:Ingredient) ON (n.name);
CREATE INDEX IF NOT EXISTS FOR (n:Keyword) ON (n.name);
CREATE INDEX IF NOT EXISTS FOR (n:DietType) ON (n.name);
CREATE INDEX IF NOT EXISTS FOR (n:Author) ON (n.name);
CREATE INDEX IF NOT EXISTS FOR (n:Collection) ON (n.name);
""" 
run_multiple_query(cq_versione5)